In [ ]:

image_path = r"<image_path>"

# get color segments clusters
from PIL import Image
import numpy as np
import cv2

from sklearn.cluster import KMeans
def quantize(image_path, n_colors=16, threshold=0.005):
    """
    Quantize image to n colors. If area of the quantized color is less than 0.5% of the total image area, it is replaced with closest color.
    """
    image = Image.open(image_path)
    image = image.convert("RGB")
    image = np.array(image)
    shape = image.shape
    image = cv2.cvtColor(image, cv2.COLOR_RGB2LAB)
    image = image.reshape((-1, 3))
    clt = KMeans(n_clusters=n_colors)
    labels = clt.fit_predict(image)
    quant = clt.cluster_centers_.astype("uint8")[labels]
    # filter colors by area
    unique, counts = np.unique(labels, return_counts=True)
    print(unique, counts)
    # sort by count
    print(sorted(zip(unique, counts), key=lambda x: x[1], reverse=True))
    quant = quant.reshape(shape).astype("uint8")
    quant = cv2.cvtColor(quant, cv2.COLOR_LAB2RGB)
    return quant

import numpy as np
import cv2
from PIL import Image
from sklearn.cluster import KMeans

def get_color_segment_areas(image_path, radius_scale=1.0, min_area=1600, min_area_ratio=0.005):
    """
    Get the areas of the color segments in the image.
    We use mask to get color-wise mask, then applies contour detection to get the area of the color segment.
    We discard the color segments whose area is less than 0.5% of the total image area.
    """
    # Open the image
    image = Image.open(image_path)
    image = image.convert("RGB")
    pixels = np.array(image)
    original_shape = pixels.shape
    pixels = pixels.reshape((-1, 3))

    # we already have quantized image, so its colors are already clustered
    colors = np.unique(pixels, axis=0) # get unique colors
    print("Unique colors:", len(colors))
    # get the color segments by masks
    masks = []
    contours_all = {tuple(color.tolist()): [] for color in colors}
    for color in colors:
        mask = np.all(pixels == color, axis=1)
        assert np.sum(mask) > 0, "Color not found in the image"
        masks.append(mask)
        #display(Image.fromarray(mask.reshape(original_shape[:-1]).astype("uint8") * 255))
    # get the area of the color segments
    for i, mask in enumerate(masks):
        # apply contour detection to get the area of the color segment
        mask = mask.reshape(original_shape[:-1])
        _, thresh = cv2.threshold(mask.astype("uint8") * 255, 127, 255, 0)
        contours, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        contours = [cnt for cnt in contours if cv2.contourArea(cnt) > max(min_area, min_area_ratio * original_shape[0] * original_shape[1])]
        # centers = [np.mean(cnt, axis=0).astype(int) for cnt in contours]
        # center can be out of actual contour, so we need to adjust it if its out of contour
        centers = [np.mean(cnt, axis=0) for cnt in contours]
        centers = [adjust_center(contour_result, center_estimate) for contour_result, center_estimate in zip(contours, centers)]
        contours = [np.squeeze(cnt) for cnt in contours]
        radiuses = [np.max(np.linalg.norm(cnt - center, axis=1)) for cnt, center in zip(contours, centers)]
        # register the color segment area
        contours_all[tuple(colors[i])] = (contours, centers, radiuses)
    valid_colors = {color: len(contours_all[color][0]) for color in contours_all}
    transparent_background = np.zeros(original_shape, dtype="uint8")
    for color in contours_all:
        if valid_colors[color] > 0:
            for cnt, center, radius in zip(contours_all[color][0], contours_all[color][1], contours_all[color][2]):
                print(center, radius)
                cv2.circle(transparent_background, tuple(center.flatten()), max(1, int(radius_scale * radius)), color, -1)
    return Image.fromarray(transparent_background)

def adjust_center(contour, center_estimate):
    """
    Adjust the center to be inside the contour.
    """
    center = center_estimate
    
    if cv2.pointPolygonTest(contour, tuple(center.flatten()), False) < 0:
        # center is outside the contour
        print("Center:", center)
        # get the closest point on the contour
        center_coords = center.flatten()
        contour_coords_as_array = contour.reshape(-1, 2)
        distances = np.linalg.norm(contour_coords_as_array - center_coords, axis=1)
        closest_point = contour[np.argmin(distances)]
        center = closest_point
        # fix to closest integer
        center = np.round(center).astype(int)
        print("Adjusted center:", center)
        assert type(center) == type(center_estimate), f"Center type mismatch: {type(center)} != {type(center_estimate)}"
    else:
        # to integer
        center = np.round(center_estimate).astype(int)
    return center
    
    
def canny_image(image_path, low=100, high=200, result_path=None):
    """
    Applies the Canny edge detection algorithm to the image.
    """
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    canny_edge = cv2.Canny(image, low, high)
    # to white background
    canny_edge = cv2.bitwise_not(canny_edge)
    if result_path:
        cv2.imwrite(result_path, canny_edge)
    return Image.fromarray(canny_edge)

def contour_image(image_path, result_path=None):
    """
    Applies the contour detection algorithm to the image.
    """
    image = cv2.imread(image_path)
    image_total_pixels = image.shape[0] * image.shape[1]
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 127, 255, 0)
    # get outmost contours only
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    # filter contours by area
    contours = [cnt for cnt in contours if cv2.contourArea(cnt) > 0.005 * image_total_pixels]
    image = cv2.drawContours(image, contours, -1, (0, 255, 0), 3)
    if result_path:
        cv2.imwrite(result_path, image)
    return Image.fromarray(image)

# quant = quantize(image_path, n_colors=16)
# quant = Image.fromarray(quant)
# quant.save("quantized.png")

# canny_quant = canny_image("quantized.png", result_path="canny_quantized.png")

# canny_quant_contour = contour_image("quantized.png", result_path="quantized_contour.png")



In [ ]:
result = (get_color_segment_areas("quantized.png", radius_scale=0.05, min_area=400, min_area_ratio=0.001))
# get count for each color
#display(result)

# result has black background, we can remove it to be transparent
result = result.convert("RGBA")
result_data = np.array(result)
# apply transparency mask where the color is black
result_data[:, :, 3] = np.where(np.all(result_data[:, :, :3] == [0, 0, 0], axis=-1), 0, 255)
result = Image.fromarray(result_data)
display(result)
# now we have filtered the color segments whose area is less than 0.5% of the total image area

# merge with the original image
original = Image.open(image_path)
original = original.convert("RGBA")

# overlay the color segments on the original image
result = result.resize(original.size)
result = result.convert("RGBA") # should be already RGBA
overlay = Image.blend(original, result, alpha=0.5)
display(overlay)

In [ ]:
# overlay the color segments on the original image
base_image = Image.open(image_path)
base_image = base_image.convert("RGBA")
result = result.convert("RGBA")
transparency_mask = Image.new("L", base_image.size, 0)
base_image.alpha_composite(result, dest=(0, 0), source=(0, 0))
base_image.convert("RGB")
display(base_image)